In [1]:
#shiny libraries
library(shiny)
library(textshaping)
library(shinyjs)
#sc libraries
library(Seurat)
library(phateR)
library(DoubletFinder)
library(monocle)
library(slingshot)
library(URD)
library(GSVA)
library(limma)
library(plyr)
library(dplyr)
library(org.Mm.eg.db)
library(org.Hs.eg.db)
library(CellChat)
library(velocyto.R)
library(SeuratWrappers)
library(stringr)
library(scran)
library(ggpubr)
library(viridis)
library(pheatmap)
library(parallel)
library(reticulate)
library(SCENIC)
library(feather)
library(AUCell)
library(RcisTarget)
library(Matrix)
library(foreach)
library(doParallel)
library(clusterProfiler)
#st libraries
library(RColorBrewer)
library(Rfast2)
library(SeuratDisk)
library(abcCellmap)
library(biomaRt)
library(copykat)
library(gelnet)
library(ggplot2)
library(parallelDist)
library(patchwork)
library(markdown)


Attaching package: ‘shinyjs’


The following object is masked from ‘package:shiny’:

    runExample


The following objects are masked from ‘package:methods’:

    removeClass, show


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Loading required package: Matrix

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, appe

In [8]:
file_list <- list.files("~/HemaScopeR/R", pattern = ".R", full.names = TRUE)

In [9]:
lapply(file_list, source)

Warning message in repr_html.shiny.tag.list(X[[i]], ...):
“Inserting HTML strings into <head> is currently not supported”


[[1]]
[[1]]$value
function (input.data.dir, output.dir, sampleName = "Hema_ST", 
    rds.file = FALSE, filename = "filtered_feature_bc_matrix.h5", 
    assay = "Spatial", slice = "slice1", filter.matrix = TRUE, 
    to.upper = FALSE, Step2_QC = TRUE, min.gene = 200, min.nUMI = 500, 
    max.gene = Inf, max.nUMI = Inf, min.spot = 0, bool.remove.mito = FALSE, 
    species = "mouse", Step3_Clustering = TRUE, normalization.method = "SCTransform", 
    npcs = 50, pcs.used = 1:10, resolution = 0.8, Step4_Find_DEGs = TRUE, 
    only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, 
    test.use = "wilcox", Step5_SVFs = TRUE, selection.method = "moransi", 
    n.top.show = 10, n.col.show = 5, Step6_Interaction = TRUE, 
    commot.signaling_type = "Secreted Signaling", commot.database = "CellChat", 
    commot.min_cell_pct = 0.05, commot.dis_thr = 500, commot.n_permutations = 100, 
    Step7_CNV = TRUE, copykat.genome = NULL, copykat.LOW.DR = 0.05, 
    copykat.UP.DR = 0.1, copykat.win.size = 25, copykat.distance = "euclidean", 
    copykat.n.cores = 1, Step8_Deconvolution = TRUE, cell2loc.sc.h5ad.dir = NULL, 
    cell2loc.sc.max.epoch = 1000, cell2loc.st.max.epoch = 10000, 
    cell2loc.use.gpu = TRUE, cell2loc.use.dataset = "LymphNode", 
    Step9_Cellcycle = TRUE, s.features = NULL, g2m.features = NULL, 
    Step10_Niche = TRUE, Nich.cluster.n = 4, condaenv = "r-reticulate", 
    verbose = FALSE, genReport = TRUE) 
{
    SpatialColors <- colorRampPalette(colors = rev(x = brewer.pal(n = 11, 
        name = "Spectral")))
    FeatureColors.bi <- colorRampPalette(colors = rev(x = brewer.pal(n = 11, 
        name = "RdYlBu")))
    FeatureColors.one <- colorRampPalette(colors = brewer.pal(n = 9, 
        name = "YlOrRd"))
    if (!dir.exists(file.path(output.dir, sampleName))) {
        dir.create(file.path(output.dir, sampleName))
    }
    else {
        warning(paste0("The new results will overwrite the file under ", 
            file.path(output.dir, sampleName)))
    }
    output.dir <- file.path(output.dir, sampleName)
    print(paste0("The results will be saved in ", output.dir))
    print("Loading data...")
    st_obj <- st_Loading_Data(input.data.dir = input.data.dir, 
        output.dir = file.path(output.dir, "Step1_Loading_Data"), 
        sampleName = sampleName, rds.file = rds.file, filename = filename, 
        assay = assay, slice = slice, filter.matrix = filter.matrix, 
        to.upper = to.upper)
    if (Step2_QC) {
        print("Performing QC of genes and spots...")
        st_obj <- QC_Spatial(st_obj = st_obj, output.dir = file.path(output.dir, 
            "Step2_QC"), min.gene = min.gene, min.nUMI = min.nUMI, 
            max.gene = max.gene, max.nUMI = max.nUMI, min.spot = min.spot, 
            species = species, bool.remove.mito = bool.remove.mito, 
            SpatialColors = SpatialColors)
    }
    if (Step3_Clustering) {
        print("Performing normalization, PCA and clustering...")
        st_obj <- st_Clustering(st_obj = st_obj, output.dir = file.path(output.dir, 
            "Step3_Clustering"), normalization.method = normalization.method, 
            npcs = npcs, pcs.used = pcs.used, resolution = resolution, 
            verbose = verbose)
    }
    if (Step4_Find_DEGs) {
        print("Finding differential expressed genes in each cluster...")
        st.markers <- st_Find_DEGs(st_obj = st_obj, output.dir = file.path(output.dir, 
            "Step4_Find_DEGs"), ident.label = "seurat_clusters", 
            only.pos = only.pos, min.pct = min.pct, logfc.threshold = logfc.threshold, 
            test.use = test.use, verbose = verbose)
    }
    if (Step5_SVFs) {
        print("Identifying spatially variable features...")
        st_obj <- st_SpatiallyVariableFeatures(st_obj = st_obj, 
            output.dir = file.path(output.dir, "Step5_SpatiallyVariableFeatures"), 
            assay = st_obj@active.assay, selection.method = selection.method, 
            n.top.show = n.top.show, n.col = n.col.show, 

In [10]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,14534477,776.3,20893922,1115.9,20893922,1115.9
Vcells,894940515,6827.9,2871554182,21908.3,2833693674,21619.4


In [ ]:
## 运行APP
shinyApp(ui = ui, 
         server = server, 
         options = list(host = "10.10.10.161", port = 9999))


Listening on http://10.10.10.161:9999

Warning message:
“CombinePlots is being deprecated. Plots should now be combined using the patchwork system.”
Warning message:
“CombinePlots is being deprecated. Plots should now be combined using the patchwork system.”
When using repel, set xnudge and ynudge to 0 for optimal results

Warning message:
“Transformation introduced infinite values in continuous x-axis”
Warning message:
“Transformation introduced infinite values in continuous x-axis”
Centering and scaling data matrix

Computing nearest neighbor graph

Computing SNN

08:34:50 UMAP embedding parameters a = 0.115 b = 1.929

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

08:34:50 Read 4939 rows and found 35 numeric columns

08:34:50 Using Annoy for neighbor search, n_neighbors = 50

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

08:34:50 Building Annoy in

In [ ]:
/public/home/rp1008csj/rp1020wangzy/HemaScopeR/test/caseData/GSE162337/GSE162337/GSE162337_Object.RDS
GSE162337
/public/home/rp1008csj/rp1020wangzy/HemaScopeR/test/GSE162337_test.temp
/public/home/rp1008csj/rp1020wangzy/anaconda3/envs/HemaScopeR/bin/python

In [ ]:
 output$vln_plot <- renderPlot({
        # 在这里生成 VlnPlot，sc_object 应该在之前的代码中被创建
        VlnPlot(sc_object, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
      })
        
       vln_plot <- VlnPlot(sc_object, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
  
      # 保存小提琴图到文件
      pdf_file <- "vln_plot.pdf"
      png_file <- "vln_plot.png"
      pdf(pdf_file, width = input$pdf_width, height = input$pdf_height)
      print(vln_plot)
      dev.off()
      png(png_file, width = input$png_width, height = input$png_height)
      print(vln_plot)
      dev.off()

      # 更新响应式值
      updated_vln_plot(list(pdf_file = pdf_file, png_file = png_file))  
      
      sc_object.temp(sc_object)
        
        output$download_pdf_button <- downloadHandler(
          filename = function() {
            "vln_plot.pdf"
          },
          content = function(file) {
            pdf_file <- updated_vln_plot()$pdf_file
            if (!is.null(pdf_file)) {
              file.copy(pdf_file, file)
            }
          }
        )

        output$download_png_button <- downloadHandler(
          filename = function() {
            "vln_plot.png"
          },
          content = function(file) {
            png_file <- updated_vln_plot()$png_file
            if (!is.null(png_file)) {
              file.copy(png_file, file)
            }
          }
        )
  })
    
    observeEvent(input$edit_button, {
      edited_color <- input$color  # 获取用户输入的颜色值

      # 在这里使用 edited_color 来重新生成小提琴图
      sc_object <- sc_object.temp()  # 获取数据
      updated_plot <- VlnPlot(sc_object, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, cols = edited_color)

      # 更新响应式值
      updated_vln_plot(list(updated_plot = updated_plot))
       
      output$vln_plot <- renderPlot({
            updated_plot
          })  
        
      # 将图形保存到文件
      pdf_file <- "vln_plot.pdf"
      png_file <- "vln_plot.png"
      pdf(pdf_file, width = input$pdf_width, height = input$pdf_height)
      print(updated_plot)
      dev.off()
      png(png_file, width = input$png_width, height = input$png_height)
      print(updated_plot)
      dev.off()

      # 更新下载链接
      output$download_pdf_button <- downloadHandler(
        filename = function() {
          "vln_plot.pdf"
        },
        content = function(file) {
          file.copy(pdf_file, file)
        }
      )

      output$download_png_button <- downloadHandler(
        filename = function() {
          "vln_plot.png"
        },
        content = function(file) {
          file.copy(png_file, file)
        }
      )
    })